In [1]:
pip install twilio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.2 MB/s eta 0:00:00


In [ ]:

import json
import os
import random
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
from datetime import datetime
import re

app = Flask(__name__)

PACIENTES_FILE = "pacientes.json"
IDIOMAS = ["es", "en"]

# Simulación básica de análisis de síntomas a especialidades
sintomas_especialidades = {
    "cabeza": "Neurología",
    "corazón": "Cardiología",
    "pecho": "Neumología",
    "dolor abdominal": "Gastroenterología",
    "huesos": "Ortopedia",
    "ojos": "Oftalmología",
    "oídos": "Otorrinolaringología",
    "sangrado": "Urgencias",
    "mareo": "Medicina interna",
    "ansiedad": "Psiquiatría",
}

# Traducción básica
traducciones = {
    "hola": {"en": "hello"},
    "¿cómo estás?": {"en": "how are you?"},
    "necesito ayuda": {"en": "i need help"},
    "me duele la cabeza": {"en": "i have a headache"},
    "emergencia": {"en": "emergency"},
}

# Cargar pacientes
def cargar_pacientes():
    if os.path.exists(PACIENTES_FILE):
        with open(PACIENTES_FILE, "r") as f:
            return json.load(f)
    return {}

# Guardar pacientes
def guardar_pacientes(data):
    with open(PACIENTES_FILE, "w") as f:
        json.dump(data, f, indent=2)

# Detección de idioma
def detectar_idioma(mensaje):
    if re.search(r'\b(hello|emergency|pain|help|please|doctor|headache|hospital)\b', mensaje.lower()):
        return "en"
    return "es"

# IA básica: identificar síntoma y sugerir especialidad
def sugerir_especialidad(mensaje):
    for sintoma, especialidad in sintomas_especialidades.items():
        if sintoma in mensaje:
            return especialidad
    return "Medicina general"

pacientes = cargar_pacientes()

@app.route("/whatsapp", methods=["POST"])
def responder():
    numero = request.form.get("From")
    mensaje = request.form.get("Body").strip().lower()
    respuesta = MessagingResponse()
    msg = respuesta.message()

    idioma = detectar_idioma(mensaje)

    if numero not in pacientes:
        pacientes[numero] = {
            "historial": [],
            "estado": "inicio",
            "idioma": idioma
        }
        guardar_pacientes(pacientes)

    usuario = pacientes[numero]

    # Agregar mensaje al historial clínico
    usuario["historial"].append({
        "timestamp": str(datetime.now()),
        "mensaje": mensaje
    })

    # IA para síntomas y especialidad
    especialidad = sugerir_especialidad(mensaje)

    if idioma == "es":
        respuesta_texto = f"🤖 Detecté que podrías necesitar atención en la especialidad de *{especialidad}*.\n¿Deseas que te agende una cita o prefieres hablar con un asesor humano?"
    else:
        respuesta_texto = f"🤖 I detected you may need attention from a *{especialidad}* specialist.\nWould you like me to schedule an appointment or connect you with a human advisor?"

    msg.body(respuesta_texto)

    guardar_pacientes(pacientes)
    return str(respuesta)

app.run(host="0.0.0.0", port=3000)
